Import packages

In [1]:
import os
import pandas as pd
import pandas.io.sql
import pyodbc
import logging

Set logging to INFO level

In [2]:
logging.basicConfig( filename='C:/Users/Mark/Documents/example.txt',level=logging.WARNING, format='%(asctime)s %(message)s') 

Set filepath to file location

In [3]:
filepath1 = 'C:/Users/Mark/Documents/DC_COVID.xlsx'
logging.info('File path is set to ' + filepath1)

filetail = os.path.basename(filepath1)
# print (filetail)

list1 = (filetail.split('.'))
filetail = (list1[0])

In [4]:
xls = pd.ExcelFile(filepath1)

In [5]:
#xls.sheet_names
#Kept here in case need to check the names of the excel sheet

Load the excel tables into two dataframes. Export them to for auditing in case of error. 
File names RawCases.csv and RawDeaths.csv

In [6]:
dfRawCases = pd.read_excel(xls, 'Total Cases by Ward', header=1)
dfRawDeaths = pd.read_excel(xls, 'Lives Lost by Ward', header=1)

dfRawCases.to_csv('C:/Users/Mark/Documents/RawCases_' + filetail +'.csv')
dfRawDeaths.to_csv('C:/Users/Mark/Documents/RawDeaths.csv')

logging.info('Files loaded and exported to csv')

Pivot Data 

In [7]:
dfCases = dfRawCases.melt(id_vars=['Ward'])
dfCases = dfCases.rename(columns={'variable': 'Date', 'value': 'Cases'})

dfDeaths = dfRawDeaths.melt(id_vars=['Ward'])
dfDeaths = dfDeaths.rename(columns={'variable': 'Date', 'value': 'Deaths'})

dfCases.to_csv('C:/Users/Mark/Documents/PivotCases.csv')
dfDeaths.to_csv('C:/Users/Mark/Documents/PivotDeaths.csv')

In [8]:
print(os.path.basename(filepath1))

DC_COVID.xlsx


Connect to local database

In [9]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-4KUL2T84\SQLEXPRESS;'
                      'Database=Covid;'
                      'Trusted_Connection=yes;')


Save SQL Query into variable to pull everything from Wards table. This will be used to map the Ward names to the Ward ID

In [10]:
SQL_Query = pd.read_sql_query(

'''
    SELECT *
    FROM Wards
'''
    , conn)

Pull Wards table in dataframe and switch the columns around. Print out to check

In [11]:
dfSQLPull = pd.DataFrame(SQL_Query)
dfSQLPull = dfSQLPull[['Ward','WardID']]
print ("SQL Table Wards")
print (dfSQLPull)

SQL Table Wards
                        Ward  WardID
0                          1       1
1                          2       2
2                          3       3
3                          4       4
4                          5       5
5                          6       6
6                          7       7
7                          8       8
8                    Unknown       9
9   Experienced Homelessness      10
10                       All      11


Change the datatype to String in order to join the dataframes. Check datatypes to confirm

In [12]:
dfSQLPull['WardID'] = dfSQLPull['WardID'].astype(str)
dfSQLPull['Ward'] = dfSQLPull['Ward'].astype(str)
dfCases['Ward'] = dfCases['Ward'].astype(str)
dfDeaths['Ward'] = dfDeaths['Ward'].astype(str)
print (dfSQLPull.dtypes)

Ward      object
WardID    object
dtype: object


Merge dataframes to map Wards to IDs

In [13]:
dfLoadRawC=pd.merge(dfCases,dfSQLPull, on="Ward")
dfLoadRawD=pd.merge(dfDeaths,dfSQLPull, on="Ward")

dfLoadRawC.to_csv('C:/Users/Mark/Documents/MergedCases.csv')
dfLoadRawD.to_csv('C:/Users/Mark/Documents/MergedDeaths.csv')


Drop the Ward Column 

In [14]:
dfLoadRawC = dfLoadRawC.drop(columns=['Ward'])
dfLoadRawD = dfLoadRawD.drop(columns=['Ward'])

Create ID Columns for both dataframes

In [15]:
dfLoadRawC['CaseID'] = dfLoadRawC.index +1
dfLoadRawD['DeathID'] = dfLoadRawD.index +1

Reorder the columns

In [16]:
dfLoadRawC = dfLoadRawC[['CaseID','Date','Cases','WardID']]

dfLoadRawD = dfLoadRawD[['DeathID','Date','Deaths','WardID']]

Insert the data into the database

In [18]:
cursor = conn.cursor()
for row in dfLoadRawD.iterrows():
    sql = 'INSERT INTO Deaths ({}) VALUES  ({})'.format(' ,'.join(dfLoadRawD.columns), ','.join(['?']*len(dfLoadRawD.columns)))
    cursor.execute(sql, tuple(row[1]))
conn.commit()

for row in dfLoadRawC.iterrows():
    sql = 'INSERT INTO Cases ({}) VALUES  ({})'.format(' ,'.join(dfLoadRawC.columns), ','.join(['?']*len(dfLoadRawC.columns)))
    cursor.execute(sql, tuple(row[1]))
conn.commit()

IntegrityError: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_Deaths'. Cannot insert duplicate key in object 'dbo.Deaths'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")

Query database to confirm and export to CSV

In [22]:
Cases_Query = pd.read_sql_query(

'''
    SELECT Cases, Ward, Date
    FROM Cases
    Inner JOIN Wards ON Cases.WardID=Wards.WardID;
'''
    , conn)
dfCasesPull = pd.DataFrame(Cases_Query)

Deaths_Query = pd.read_sql_query(

'''
    SELECT Deaths, Ward, Date
    FROM Deaths
    Inner JOIN Wards ON Deaths.WardID=Wards.WardID;
'''
    , conn)
dfDeathsPull = pd.DataFrame(Deaths_Query)

Cases_Query.to_csv('C:/Users/Mark/Documents/SQLCases.csv')
Deaths_Query.to_csv('C:/Users/Mark/Documents/SQLDeaths.csv')